# Monthly Retrospective script
The following script will produce a list of statistics that might be relevant for the Monthly retrospective. **This new version uses the Geonet API and can therefore be run on the first!**

All you need to do is change the dates to the correct month in the box below and select "run all cells".

This is a work in progress and would appreciate any feedback as to what other information, graphs etc would be useful. 

 **Note the felt reports need to be manually checked as sometimes aftershocks or unrelated events associate felt reports from different quakes**

In [ ]:
#modules to import
import requests
import pandas as pd

## Times
Change the date here. Because it is in UTC time, start it on the last day of the previous month and end it on the last of the current month. 
During daylight savings time (Summer), midnight NZT is at at 11:00:00, otherwise set it to 12:00:00

In [ ]:
starttime = "2019-12-31T11:00:00" #change the 12:00:00 for daylight savings. Be sure to leave the T.
endtime = "2020-01-31T11:00:00"

The following section is the code that will automatically run with the above input. The output is found at the bottom.

In [ ]:
def monthlyretro(starttime, endtime):
    
    """Prints a summary of magnitudes of a month's quakes, as well as the SOP defined location 
       and the top 10 most felt earthquakes"""
    
    #The first function is a JSON file which contains earthquake information. 
    #The query is generated through the WFS service off Geonet - https://wfs.geonet.org.nz/
    #the inputs starttime and endtime define the time boundaries
    quakejson = requests.get(
        "http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request="
        "GetFeature&typeName=geonet:quake_search_v1&outputFormat=json&cql_filter=origintime>=" 
    + starttime + "+AND+origintime<=" + endtime).json()
    
    lat, lon, mag, quakeid, counts, url = [[] for x in range(6)] #empty lists to iterate to
    
    #the following is a for loop which populates above lists with values from JSON
    for a in range(len(quakejson['features'])):
        lon.append(quakejson['features'][a]['geometry']['coordinates'][0])
        lat.append(quakejson['features'][a]['geometry']['coordinates'][1])
        mag.append(quakejson['features'][a]['properties']['magnitude'])
        quakeid.append(quakejson['features'][a]['properties']['publicid'])
        
    #dataframe containing above information
    df = pd.DataFrame({"latitude":lat,"longitude":lon,"magnitude":mag,"quake_id":quakeid})
    
    #empty Dataframe for following functions
    summary_df = pd.DataFrame({})
    
    #setting bins for magnitude summary
    summary_df['magnitudes'] = ["0-1", "1-2","2-3","3-4","4-5","5-6","6-7","7-8","8+"]
    
    #counting magnitude by bin
    summary_df['count'] = (df['magnitude'].value_counts(bins = [0,1,2,3,4,5,6,7,8,9]))
    summary_df = summary_df.sort_index()
    
    #setting index as bins defined above
    summary_df = summary_df.set_index('magnitudes')

    print(summary_df)
    #count number of events within Kermadec SOP zone
    kermadec = df[(df['latitude'] <=-25) & (df['latitude']>=-33) & 
                  (df['longitude'] >= 177) & (df['longitude'] <=180) | 
                  (df['latitude'] <=-25) & (df['latitude']>=-33) & 
                  (df['longitude'] >= -180) & (df['longitude'] <=-169)]

    #count number of events within Local SOP zone
    local = df[(df['latitude'] <=-33) & (df['latitude'] >= -50) & 
               (df['longitude'] >= 163) & (df['longitude'] <=180) | 
               (df['latitude'] <=-33) & (df['latitude'] >= -50) & 
               (df['longitude'] >= -180) & (df['longitude'] <=-175)|
               (df['latitude'] <=-32) & (df['latitude'] >= -33) & 
               (df['longitude'] >= 163) & (df['longitude'] <=177)]

    #print results, tele calculated as total minue kermadec and local quakes
    print("\nTotal quakes: ", len(df), "\nLocal", len(local), "\nKermadec: ", 
          len(kermadec), "\nTeleseismics: ", len(df) - len(kermadec) - len(local),
         "\nMaxmimum magnitude", df['magnitude'].max(), " ID: ", 
          df.iloc[df['magnitude'].idxmax()]['quake_id'], "\nMinimum magnitude",
          df['magnitude'].min(), "ID: ", df.iloc[df['magnitude'].idxmin()]['quake_id'])

    #felt reports section
    for a in range(len(quakeid)):
        url.append('http://api.geonet.org.nz/intensity?type=reported&publicID=' + quakeid[a])
    felt_reports = {}
    for a in range(len(url)):
        felt_reports[a] = requests.get(url[a]).json()
    for a in felt_reports:
        counts.append(felt_reports[a]['count'])
    felt_df = pd.DataFrame({"Felt Reports":counts}, index = quakeid)
    felt_df = felt_df.sort_values(by='Felt Reports', ascending = False)
    #prints top ten felt quakes
    print("\n", felt_df.head(n=10))

In [ ]:
monthlyretro(starttime,endtime)